In [ ]:
!pip install kaggle
from google.colab import files
files.upload()

In [ ]:
!mkdir ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [ ]:
!kaggle datasets download -d mkechinov/ecommerce-behavior-data-from-multi-category-store -p /content/data
!unzip /content/data/ecommerce-behavior-data-from-multi-category-store.zip -d /content/data


In [ ]:
DATA_DIR = "/data"


In [ ]:
import os, pandas as pd

DATA_DIR = "/content/data"
csv_files = [os.path.join(DATA_DIR, f) for f in os.listdir(DATA_DIR) if f.endswith(".csv")]
print("الملفات الموجودة:", csv_files)

sample = pd.read_csv(csv_files[0], nrows=5)
print("أعمدة البيانات:")
print(sample.columns.tolist())
sample.head()


🔹 الخطوة 2: تحميل البيانات الضخمة على دفعات (chunks) لتجنّب نفاد الذاكرة

In [ ]:
import pandas as pd
import gc
from collections import Counter
import time

file_path = "/content/data/2019-Oct.csv"

chunksize = 2_000_000
event_counter = Counter()
start = time.time()

for chunk in pd.read_csv(file_path, chunksize=chunksize, low_memory=False):
    event_counter.update(chunk["event_type"].values)
    del chunk
    gc.collect()

end = time.time()
print("أكثر الأحداث تكراراً:")
print(event_counter.most_common())
print(f"الوقت المستغرق: {end - start:.2f} ثانية")


# الخطوة 3: تحليل بسيط — أكثر العلامات التجارية (brands) تكراراً لكل نوع حدث

In [ ]:
import pandas as pd
import gc

file_path = "/content/data/2019-Oct.csv"
chunksize = 1_000_000

result = {}

for chunk in pd.read_csv(file_path, chunksize=chunksize, low_memory=False):
    for event_type, df_group in chunk.groupby("event_type"):
        top_brands = (
            df_group["brand"].value_counts().head(3).to_dict()
        )
        if event_type not in result:
            result[event_type] = Counter()
        result[event_type].update(top_brands)
    del chunk
    gc.collect()

for event_type, brands in result.items():
    print(f"\nالحدث: {event_type}")
    print("أشهر العلامات التجارية:", dict(brands.most_common(5)))


# الخطوة 4: تحويل البيانات إلى Parquet (اختياري  لتسريع التحليل )

In [ ]:
import dask.dataframe as dd
import time

start = time.time()
ddf = dd.read_csv("/content/data/*.csv", assume_missing=True, blocksize="64MB")
ddf.to_parquet("/content/data/parquet_data", engine="pyarrow", compression="snappy")
end = time.time()

print(f"تم التحويل إلى Parquet خلال {end - start:.1f} ثانية.")


#  الخطوة 5: تحليل باستخدام Dask (على جميع الملفات دفعة واحدة)

In [ ]:
import dask.dataframe as dd
from dask.diagnostics import ProgressBar

ddf = dd.read_csv("/content/data/*.csv", assume_missing=True, blocksize="64MB")
print("أسماء الأعمدة:", list(ddf.columns))

with ProgressBar():
    event_counts = ddf["event_type"].value_counts().compute()

print("\nعدد الأحداث:")
print(event_counts)

with ProgressBar():
    avg_price = ddf.groupby("event_type")["price"].mean().compute()

print("\nمتوسط السعر لكل نوع حدث:")
print(avg_price)


# 1. توزيع الأحداث على مدار اليوم (سلوك المستخدم)

In [ ]:
import dask.dataframe as dd
from dask.diagnostics import ProgressBar

ddf = dd.read_parquet("/content/data/parquet_data")

ddf["event_time"] = dd.to_datetime(ddf["event_time"], errors="coerce")
ddf["hour"] = ddf["event_time"].dt.hour

with ProgressBar():
    hourly = ddf.groupby("hour")["event_type"].value_counts().compute().unstack(fill_value=0)

hourly.plot(kind="line", figsize=(10,5), title="توزيع الأحداث حسب الساعة (view/cart/purchase)")


# 2. مقارنة الأسعار بين العلامات التجارية في المشتريات فقط

In [ ]:
import pandas as pd

with ProgressBar():
    purchase_df = ddf[ddf["event_type"] == "purchase"]
    brand_avg_price = purchase_df.groupby("brand")["price"].mean().compute().sort_values(ascending=False).head(10)

print("أغلى 10 علامات تجارية في المشتريات:")
print(brand_avg_price)


# 3. نسبة التحويل (Conversion Rate)
أي: كم من مشاهدات المنتجات تتحول إلى عمليات شراء؟

In [ ]:
with ProgressBar():
    views = ddf[ddf["event_type"] == "view"]["product_id"].nunique().compute()
    purchases = ddf[ddf["event_type"] == "purchase"]["product_id"].nunique().compute()

conversion_rate = (purchases / views) * 100
print(f"نسبة التحويل من مشاهدة إلى شراء: {conversion_rate:.4f}%")


# 4. تحليل العلامات التجارية الأكثر جذبًا (بناءً على معدل الشراء من المشاهدة)

In [ ]:
with ProgressBar():
    views_per_brand = ddf[ddf["event_type"] == "view"].groupby("brand")["user_id"].count().compute()
    purchases_per_brand = ddf[ddf["event_type"] == "purchase"].groupby("brand")["user_id"].count().compute()

brand_conversion = (purchases_per_brand / views_per_brand * 100).dropna().sort_values(ascending=False).head(10)
print("أكثر العلامات التجارية من حيث معدل التحويل:")
print(brand_conversion)


CSV

In [ ]:
import os
import dask.dataframe as dd
import pandas as pd
from dask.diagnostics import ProgressBar


DATA_PARQUET = "/content/data/parquet_data"
RESULTS_DIR = "/content/results"
os.makedirs(RESULTS_DIR, exist_ok=True)

print("donload data ")
ddf = dd.read_parquet(DATA_PARQUET)




ddf["event_time"] = dd.to_datetime(ddf["event_time"], errors="coerce")






with ProgressBar():
    event_counts = ddf["event_type"].value_counts().compute()
event_counts.to_csv(f"{RESULTS_DIR}/event_counts.csv", header=["count"])
print(" event_counts.csv  ")






with ProgressBar():
    avg_price = ddf.groupby("event_type")["price"].mean().compute()
avg_price.to_csv(f"{RESULTS_DIR}/avg_price_per_event.csv", header=["avg_price"])
print(" avg_price_per_event.csv  ")





with ProgressBar():
    top_brands = (
        ddf.groupby(["event_type", "brand"])["user_id"]
        .count()
        .compute()
        .reset_index()
        .rename(columns={"user_id": "count"})
    )
top_brands = (
    top_brands.sort_values(["event_type", "count"], ascending=[True, False])
    .groupby("event_type")
    .head(10)
)
top_brands.to_csv(f"{RESULTS_DIR}/top_brands_by_event.csv", index=False)
print("✔ top_brands_by_event.csv  ")





ddf["hour"] = ddf["event_time"].dt.hour
with ProgressBar():
    hourly = ddf.groupby("hour")["event_type"].value_counts().compute().unstack(fill_value=0)
hourly.to_csv(f"{RESULTS_DIR}/hourly_event_distribution.csv")
print(" hourly_event_distribution.csv  ")





with ProgressBar():
    views = ddf[ddf["event_type"] == "view"]["product_id"].nunique().compute()
    purchases = ddf[ddf["event_type"] == "purchase"]["product_id"].nunique().compute()
conversion_rate = (purchases / views) * 100
pd.DataFrame(
    {"views": [views], "purchases": [purchases], "conversion_rate(%)": [conversion_rate]}
).to_csv(f"{RESULTS_DIR}/conversion_rate.csv", index=False)
print(" conversion_rate.csv  ")





with ProgressBar():
    views_per_brand = ddf[ddf["event_type"] == "view"].groupby("brand")["user_id"].count().compute()
    purchases_per_brand = ddf[ddf["event_type"] == "purchase"].groupby("brand")["user_id"].count().compute()

brand_conversion = (
    (purchases_per_brand / views_per_brand * 100)
    .dropna()
    .sort_values(ascending=False)
    .reset_index()
)
brand_conversion.columns = ["brand", "conversion_rate(%)"]
brand_conversion.to_csv(f"{RESULTS_DIR}/brand_conversion_rate.csv", index=False)
print(" brand_conversion_rate.csv تم حفظه")


print("\n تم إنشاء جميع ملفات النتائج في:", RESULTS_DIR)
print("المخرجات:")
for f in os.listdir(RESULTS_DIR):
    print(" -", f)


# إنشاء المقارنة النهائية

In [ ]:
import os, time, gc, pandas as pd
import dask.dataframe as dd
import matplotlib.pyplot as plt
from dask.diagnostics import ProgressBar

DATA_DIR = "/content/data"
RESULTS_DIR = "/content/results"
os.makedirs(RESULTS_DIR, exist_ok=True)

def dir_size(path):
    if os.path.isfile(path):
        return os.path.getsize(path)
    total = 0
    for dirpath, _, filenames in os.walk(path):
        for f in filenames:
            total += os.path.getsize(os.path.join(dirpath, f))
    return total


import pandas as pd
from collections import Counter

pandas_start = time.time()
event_counter = Counter()
chunksize = 2_000_000
file_path = os.path.join(DATA_DIR, "2019-Oct.csv")

for chunk in pd.read_csv(file_path, chunksize=chunksize, low_memory=False):
    event_counter.update(chunk["event_type"].values)
    del chunk
    gc.collect()

pandas_end = time.time()
pandas_time = pandas_end - pandas_start
pandas_size = dir_size(file_path)

print(f" pandas + chunksize done in {pandas_time:.2f} s")


dask_start = time.time()
with ProgressBar():
    ddf = dd.read_csv(os.path.join(DATA_DIR, "*.csv"), assume_missing=True, blocksize="64MB")
    _ = ddf["event_type"].value_counts().compute()
dask_end = time.time()
dask_time = dask_end - dask_start
dask_size = dir_size(DATA_DIR)
print(f" Dask done in {dask_time:.2f} s")


parquet_dir = os.path.join(DATA_DIR, "parquet_data_auto")
parquet_start = time.time()
ddf.to_parquet(parquet_dir, engine="pyarrow", compression="snappy", write_index=False)
parquet_end = time.time()
parquet_time = parquet_end - parquet_start
parquet_size = dir_size(parquet_dir)
print(f" Parquet (compressed) done in {parquet_time:.2f} s")

comparison = pd.DataFrame({
    "الطريقة": ["pandas + chunksize", "Dask", "Parquet (compressed)"],
    "الزمن المستغرق (ثانية)": [pandas_time, dask_time, parquet_time],
    "حجم البيانات (MB)": [pandas_size/1e6, dask_size/1e6, parquet_size/1e6],
    "المزايا": [
        "تعمل على أي جهاز - لا تسبب Out of Memory",
        "تتعامل مع بيانات ضخمة بسهولة وسرعة عالية",
        "تحسن التخزين وتسرع القراءة المستقبلية"
    ],
    "العيوب": [
        "تحتاج إلى حلقة for وتكرار يدوي",
        "تحتاج إلى مكتبة إضافية Dask",
        "تستهلك وقت أولي كبير للتحويل"
    ]
})

comparison.to_csv(f"{RESULTS_DIR}/methods_comparison.csv", index=False, encoding='utf-8-sig')
print("\n تم حفظ جدول المقارنة في:", f"{RESULTS_DIR}/methods_comparison.csv")


plt.figure(figsize=(7,4))
plt.bar(comparison["الطريقة"], comparison["الزمن المستغرق (ثانية)"], color=['skyblue','orange','green'])
plt.title("مقارنة زمن التنفيذ بين الطرق الثلاث")
plt.ylabel("الزمن (ثانية)")
plt.xticks(rotation=15)
plt.tight_layout()
plt.savefig(f"{RESULTS_DIR}/methods_comparison.png")
plt.show()


print(" تم حفظ الرسم البياني  في:", f"{RESULTS_DIR}/methods_comparison.png")

display(comparison)